In [29]:
import numpy as np
import numpy.random as npr
import cvxpy as cp

### Define Functions

In [51]:
def greedy_lower_bound(K, w, r):
    m,n = K.shape
    K_bar = 1-K
    v = np.zeros(n)
    for i in range(n):
        v[i] = np.ones(m) @ K_bar[:, i] * w[i]

    v_srtd = np.flip(np.argsort(v))
    v[v_srtd]

    # v_srtd is the arguments of the "most important" features to pick

    x = np.zeros(n)
    for i in range(n):
        x_temp = np.copy(x)
        x_temp[v_srtd[i]] = 1
        if np.count_nonzero(K @ x_temp) <= m*(1-r):
            x = x_temp

    # COUNT IS LOWER BOUND
    count = x @ w
    return count, x

In [52]:
def solve_upper_bound(K, w, r):
    m,n = K.shape
    K_bar = 1 - K
    x = cp.Variable(n, boolean = True)
    obj = cp.Maximize(w @ x)
    # cons = [cp.norm(K @ x, p = 0) <= m * (1-r)]
    cons = [r * m * cp.sum(x) - np.ones(m) @ K_bar @ x <= 0]

    prob = cp.Problem(obj, cons)
    prob.solve()
    return prob.value, x.value

In [72]:
def sdp_relaxation(K,w,r):
    m,n = K.shape
    X = cp.Variable((n+m+1,n+m+1), PSD = True)

    obj = cp.Maximize(w @ X[n+m,:n])
    cons = []
    cons += [cp.sum(X[n+m,n:n+m]) >= r*m]

    for i in range(n):
        cons += [X[i,i] == X[n+m,i]]
    for j in range(n, n+m):
        cons += [X[j,j] == X[n+m,j]]
    cons += [X[m+n,m+n] == 1]
    for i in range(n):
        for j in range(m):
            if K[j,i] == 1:
                cons += [X[i,n+j] == 0]

    prob = cp.Problem(obj, cons)

    prob.solve()
    return prob.value, X.value

In [73]:
# is the actual constraint satisfied
def true_const(K, r, x):
    m,_ = K.shape
    return np.count_nonzero(K @ x) <= m*(1-r)

## Make random data matrix

In [77]:
npr.seed(0)

m = 20
n = 10
k = 0

D = npr.randint(2, size = (m,n))
K = np.abs(D - D[k,:])
K_bar = 1-K

# information weighting vector
w = npr.randint(0, 10, size = n)
r = 0.6

In [75]:
p_val_low, x_val_low = greedy_lower_bound(K,w,r)
p_val_high, x_val_high = solve_upper_bound(K,w,r)

bound_tight = true_const(K, x_val_high,x_val_high)

p_val_high_sdp, x_val_high_sdp = sdp_relaxation(K,w,r)

In [76]:
print("Upper bound LP = ", p_val_high)
print("Lower_bound = ", p_val_low)
print("upper bound sdp = ", p_val_high_sdp)
print("upper bound LP tight = ", true_const(K,r, x_val_high))


Upper bound LP =  8.0
Lower_bound =  8.0
upper bound sdp =  18.526315793344544
upper bound LP tight =  True
